In [6]:
import os
import sys
sys.path.append('../../..')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import joblib
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import feature_column as fc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn import preprocessing
from sklearn.utils import shuffle
from tensorflow.keras.models import save_model,load_model

from utils.util import *
from utils.preprocessing import *
from utils.dataiter import Dataiter
from utils.evaluate import calculate_ctr, compute_rce, average_precision_score
from utils.target_encode import MTE_one_shot

import tensorflow.keras.backend as K
import core.config as conf

In [7]:
train = pd.read_csv('train2.csv', names=['text_tokens', 'creator_id', 'engager_id', 'tweet_id', 'tweet_type',
       'language', 'creator_account_creation', 'creator_follower_count',
       'creator_following_count', 'engager_follower_count',
       'engager_following_count', 'domains', 'media', 'tweet_timestamp',
       'reply', 'retweet', 'comment', 'like', 'dt_day', 'dt_dow', 'dt_hour',
       'len_domains', 'creator_main_language', 'engager_main_language',
       'creator_and_engager_have_same_main_language',
       'is_tweet_in_creator_main_language',
       'is_tweet_in_engager_main_language',
       'engager_feature_number_of_previous_reply_engagement',
       'creator_feature_number_of_previous_reply_engagement',
       'number_of_engagements_positive',
       'creator_number_of_engagements_positive',
       'engager_number_of_engagements_ratio_reply',
       'creator_number_of_engagements_ratio_reply'], header=None)

In [8]:
train.head()

,text_tokens,creator_id,engager_id,tweet_id,tweet_type,language,creator_account_creation,creator_follower_count,creator_following_count,engager_follower_count,...,engager_main_language,creator_and_engager_have_same_main_language,is_tweet_in_creator_main_language,is_tweet_in_engager_main_language,engager_feature_number_of_previous_reply_engagement,creator_feature_number_of_previous_reply_engagement,number_of_engagements_positive,creator_number_of_engagements_positive,engager_number_of_engagements_ratio_reply,creator_number_of_engagements_ratio_reply
0,101\t147\t69849\t11447\t57277\t63310\t136\t106...,F09233A58769507FD4E6FD618BCFA5B6,CFFD4D0D3AAF28B5397A98E1F4C92C77,6238B9E15E83B6D477394E9D80B3784E,3,10,1525021638,560,645,595,...,35,0,0,0,0,7,472,48,0.000000,0.145833
1,101\t56898\t137\t10799\t11039\t168\t26978\t131...,46BEEE566BB708F18075A60FDDAA8503,E0F9A3BB331C7B613B165F32F8D273BE,731FB90C6CFEF1B71D322106573F71DB,0,1,1348805977,2591,3690,514,...,1,1,1,1,0,0,3,14,0.000000,0.000000
2,101\t13304\t179\t14902\t74788\t10390\t33993\t1...,83F59BF2E0778AC4078FA3F7B71F5960,58ABF7FB24D65FB94C6487915A889460,89276E5272498E858EE8AF691EBF0951,3,5,1583939938,174,146,376,...,4,0,1,0,0,0,12,7,0.000000,0.000000
3,101\t620\t55399\t17451\t69304\t82692\t572\t105...,C96110509F8B6256CE08A8AE85ED75DB,76730357A84D1D04A662FDB1449F1AFF,502FDBC0EB4E7AB157D38262817716EB,3,12,1306675315,115852,2124,22461,...,1,0,1,0,24,1,486,20,0.049383,0.050000
4,101\t10747\t12723\t10124\t29956\t91518\t10142\...,2031F18622E7C6A30E702D33776BF2C1,9008C5AB3CB46623ADBD4C68A2A2D088,CF1F523F7D4D4139E5FD3EBD72F27D5A,3,0,1308377880,1208919,1101,388,...,13,0,1,0,4,66,35,2331,0.114286,0.028314


## one-hot encoding

In [9]:
tweet_type = train['tweet_type']

In [22]:
num = np.unique(tweet_type, axis=0)
num = np.array([0, 1, 2, 3])
num = num.shape[0]

encoding = np.eye(num)[tweet_type]

In [36]:
train['tweet_type'] = encoding

In [50]:
pd.Series(encoding).map(lambda x: ''.join(x))

Exception: Data must be 1-dimensional

In [32]:
train['a'] = encoding 

In [38]:
train

,text_tokens,creator_id,engager_id,tweet_id,tweet_type,language,creator_account_creation,creator_follower_count,creator_following_count,engager_follower_count,...,creator_and_engager_have_same_main_language,is_tweet_in_creator_main_language,is_tweet_in_engager_main_language,engager_feature_number_of_previous_reply_engagement,creator_feature_number_of_previous_reply_engagement,number_of_engagements_positive,creator_number_of_engagements_positive,engager_number_of_engagements_ratio_reply,creator_number_of_engagements_ratio_reply,a
0,101\t147\t69849\t11447\t57277\t63310\t136\t106...,F09233A58769507FD4E6FD618BCFA5B6,CFFD4D0D3AAF28B5397A98E1F4C92C77,6238B9E15E83B6D477394E9D80B3784E,0.0,10,1525021638,560,645,595,...,0,0,0,0,7,472,48,0.000000,0.145833,0.0
1,101\t56898\t137\t10799\t11039\t168\t26978\t131...,46BEEE566BB708F18075A60FDDAA8503,E0F9A3BB331C7B613B165F32F8D273BE,731FB90C6CFEF1B71D322106573F71DB,1.0,1,1348805977,2591,3690,514,...,1,1,1,0,0,3,14,0.000000,0.000000,1.0
2,101\t13304\t179\t14902\t74788\t10390\t33993\t1...,83F59BF2E0778AC4078FA3F7B71F5960,58ABF7FB24D65FB94C6487915A889460,89276E5272498E858EE8AF691EBF0951,0.0,5,1583939938,174,146,376,...,0,1,0,0,0,12,7,0.000000,0.000000,0.0
3,101\t620\t55399\t17451\t69304\t82692\t572\t105...,C96110509F8B6256CE08A8AE85ED75DB,76730357A84D1D04A662FDB1449F1AFF,502FDBC0EB4E7AB157D38262817716EB,0.0,12,1306675315,115852,2124,22461,...,0,1,0,24,1,486,20,0.049383,0.050000,0.0
4,101\t10747\t12723\t10124\t29956\t91518\t10142\...,2031F18622E7C6A30E702D33776BF2C1,9008C5AB3CB46623ADBD4C68A2A2D088,CF1F523F7D4D4139E5FD3EBD72F27D5A,0.0,0,1308377880,1208919,1101,388,...,0,1,0,4,66,35,2331,0.114286,0.028314,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4338901,101\t1972\t18825\t3425\t100\t14120\t131\t120\t...,968FAF2F52AACA58C2B41CE5B13A811D,1BC672DA2F063F2DE03DA89BA6B621C5,EE63068E74D7E293457FDC2EA3527D65,0.0,1,1359315180,248,315,1785,...,1,1,1,0,1,38,30,0.000000,0.033333,0.0
4338902,101\t56898\t137\t25944\t22659\t11305\t131\t181...,54652625625729AE0E38196970594938,39D3029E8FDC3A18472D28436EC9E776,F15B4615DA12447331AFEC90FA814B4D,1.0,0,1293912925,114,937,183,...,1,1,1,1,0,69,0,0.014493,0.000000,1.0
4338903,101\t100\t108\t16062\t11281\t10115\t11274\t102...,614E269F8E5D731946BDBC86E9DC4F4D,4A4E526A7390A5CC9D02ECB5CB62F377,DA947A6368CF4414BEA9172003A639FD,0.0,3,1512095330,56033,121,58,...,0,0,1,0,39,1,3436,0.000000,0.011350,0.0
4338904,101\t100\t27577\t30416\t10107\t10655\t11239\t1...,5A5C7CCA69CDEBBD7E81C3797DE9DD7E,D8CAFE1402CE86E007370664FC553C2B,730879D11261FA6700804AD49CCB59BB,0.0,0,1251066468,6745,368,2,...,1,1,1,0,0,4,14,0.000000,0.000000,0.0
